# This notebook launches the LatissAcquireAndTakeSequence SAL script,
### The script is built for the scriptQueue but can be run via this notebook

#### This script is used to both manually acquire a PNe target, then take data with two different gratings

In [ ]:
import sys
import os
import asyncio
import time

import numpy as np
import logging 
import yaml
import astropy

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

In [ ]:
# Derive and index for the script that is specific to your user
print(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
print(f'The generated index is {index}')

In [ ]:
# Instantiate the script then start all remotes
script = LatissAcquireAndTakeSequence(index=index)
await script.start_task

In [ ]:
# set the script to have a DEBUG log level
script.log.level = logging.DEBUG

## Declare the Target

In [ ]:
target = 'PNG321.0+03.9 '
# coordinates required if not a SIMBAD object
object_ra = '14:59:53'
object_dec = '-54:18:07'

In [ ]:
# Print the available instrument filter and gratings
# this is useful both for slewing and for the configuration steps below
inst_setup = await script.latiss.get_available_instrument_setup()
print(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

# Performing the acquisition ONLY

### Note the below configuration just slews to the target and does not try to iterate as we're not sure it's the brightest target in the field

In [ ]:
# The configurable parameters can be seen from either inside the script or from uncommenting the following command
# script.get_schema()

In [ ]:
# Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking.
# The position of the target on the detector is related to the grating.
configuration = yaml.safe_dump({"object_name": target,
                                "object_ra": object_ra,                                
                                "object_dec": object_dec,
                                "do_acquire": True,
                                "acq_filter": 'FELH0600',
                                "acq_grating": 'empty_1',
                                "acq_exposure_time": 2.0,
                                "target_pointing_tolerance": 3.0,       # Target positioning tolerance (arcsec)
                                "max_acq_iter": 0,                     # Maximum number of iterations to perform when acquiring
                                "target_pointing_verification": True,  # Takes an additional image showing the final target position
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                               })
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

await script.arun()

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()

In [ ]:
# Now manually put the target on 1750,300
current_pos = np.array((1640,1820))# x,y from firefly monitor
target_pos = (1750,300)

pixel_scale = 0.09569  # arcsec/pixel
offset = (current_pos - target_pos) * pixel_scale

print(offset)

In [ ]:
await script.atcs.offset_xy(x=offset[0], y=offset[1])

In [ ]:
# verify offset is correct
await script.latiss.take_engtest(2)

## Performing the science data collection ONLY with ronchi170lpmm

In [ ]:
# The configurable parameters can be seen from either inside the script or from uncommenting the following command
# script.get_schema()

In [ ]:
# Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking.
# The position of the target on the detector is related to the grating.
configuration = yaml.safe_dump({"object_name": target,
                                "object_ra": object_ra,                                
                                "object_dec": object_dec,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [10, 10, 10, 30,
                                                            30, 30, 60, 60,
                                                            60], 
                                "grating_sequence": ['ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'ronchi170lpmm'],
                                "filter_sequence": ['empty_1','empty_1', 'empty_1', 'empty_1',
                                                    'empty_1','empty_1', 'empty_1','empty_1',
                                                    'empty_1'], 
                                })
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

await script.arun()

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()

## Performing the science data collection ONLY with holo4_003

In [ ]:
# The configurable parameters can be seen from either inside the script or from uncommenting the following command
# script.get_schema()

In [ ]:
# Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking.
# The position of the target on the detector is related to the grating.
configuration = yaml.safe_dump({"object_name": target,
                                "object_ra": object_ra,                                
                                "object_dec": object_dec,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [10, 10, 10, 30,
                                                            30, 30, 60, 60,
                                                            60], 
                                "grating_sequence": ['holo4_004','holo4_003','holo4_003','holo4_003',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',
                                                    'holo4_003'],
                                "filter_sequence": ['empty_1','empty_1', 'empty_1', 'empty_1',
                                                    'empty_1','empty_1', 'empty_1','empty_1',
                                                    'empty_1'], 
                                })
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

await script.arun()

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()